In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("college_data.csv")

# Problem 1

In [3]:
from sympy import *
from numpy import linalg as LA
from sympy import Matrix
#found this last one on https://stackoverflow.com/questions/5889142/python-numpy-scipy-finding-the-null-space-of-a-matrix

In [4]:
#line search to ensure that alpha is not too small
def Check_Alpha(x, mu, p, eval_grad):
    '''
    Theory-based dynamic alpha value calculator. This finds the alpha value via a line search.
    Inputs:
        x - point (x,y tuple)
        mu - multiplier that acts as alpha when iterating to decide final alpha value (f(x2) = x1 + mu*p(x))
        p - negative gradient vector of f at x
        eval_grad - gradient function passed from MakeGraph function that takes the gradient of a vector
    Outputs:
        Returns appropriate alpha (step change) based on inputs
    '''
    i = 0  #initialize
    alpha = 2**(-i) #multiplier
    x_new = x+alpha*p  #initializes while loop
    gradf = np.array(eval_grad(x[0],x[1])) #gradient
    #While the function is greater than the function at the point is greater than the function + mu* alpha* np.matmul(p, gradf)
    while np.array(eval_f(x_new[0],x_new[1])) > np.array(eval_f(x[0],x[1]))+ mu* alpha* np.matmul(p, gradf):
        i += 1
        alpha = 2**(-i)
        x_new = x + alpha* p 
    return alpha

In [5]:
def Steepest_Descent(x, p, gradf, tol, eval_grad):
    '''
    This is our algorithm! In very general terms, this function takes a function, a start point, and a tolerance level, and, using the 
    Steepest Descent algorithm, will figure out where the start point coverges to. 
    Inputs:
        x - point (x,y tuple) 
        p - negative gradient vector of f at x
        gradf - gradient vector of f
        tol - how close do we need to be to the to the final point to get the algorithm to stop
        eval_grad - gradient function passed from MakeGraph function that takes the gradient of a vector
    Outputs:
        Returns a list of tuples (x,y) that represent the points the algorithm travels to as it converges
    '''
    x_values=[]
    x_values.append(x)
    while LA.norm(gradf)>tol:
        #find my alpha via line search
        alpha = Check_Alpha(x, .5, p, eval_grad)

        x = x+alpha*p #update x
        #update gradient with new x
        gradf = np.array(eval_grad(x[0],x[1]))
        p = -gradf
        #Return your x values and append all them so you are showing all of them. See prompt
        x_values.append(x)
    return x_values

In [6]:
def Make_Graph():
    '''
    Inputs:
        No parameters
    Outputs:
        This is a function designed to bring everything together! By running this function we should have a contour plot that
        plots each point the Steepest Descent algorithm goes to as it converges. Essentially this is just plotting the list of tuples(x,y)
        returned from the function Steepest_Descent onto a contour plot
    '''
    grad = derive_by_array(f, (x,y))
    #Evaluate the gradient
    eval_grad = lambdify([x,y],grad)
    #Continue evaluating the gradient
    gradf = np.array(eval_grad(x0[0],x0[1]))
    p = -gradf
    #This creates a list of the x values as we run steepest descent
    values = Steepest_Descent(x0,p,gradf, 3, eval_grad)
    #set up x1 and x2 values:
    values = np.array(values) #convert to numpy array
    for i in range(len(values)):
        plt.scatter(values[i][0], values[i][1], s = 20, color = 'xkcd:ocean blue')
        plt.text(values[i][0], values[i][1]+.1, f"{i}", color = 'r')
    x_ = np.linspace(-4.5, 4.5, 100)
    plt.xlim(-4,4)
    y_ = np.linspace(-4.5, 4.5, 100)
    plt.ylim(-4,4)
    X,Y = np.meshgrid(x_, y_) #creates all possibilities for x and y on which to plot the 3rd dimension
    Z = eval_f(X,Y)
    cp = plt.contour(X, Y, Z) #plots the contour map
    plt.clabel(cp, inline=1, fontsize=10) #labels the level curves
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

In [7]:
x,y = symbols('x y', real = True);
eval_f = lambda x,y: -10*x**2*y+5*x**2+4*y**2+x**4+2*y**4 #for the purpose of evaluating the function strictly in python
f = -10*x**2*y+5*x**2+4*y**2+x**4+2*y**4 #using sympy to make the same function
x0 = np.array([-3,0])

In [8]:
Make_Graph()

# Problem 2

### Part 1

In [9]:
def histogram(ser = data['Apps'], bin_width = 2000, tail = .009):
    '''
    Prints and displays a PDF table and graph for the input data
    Inputs:
        ser - a Pandas Series (or DataFrame column) of numerical data
        bin_width - range to count observations into one bar
        tail - percentage of data that is to the right of the distribution (as a decimal)
    Outputs:
        Does not return anything
        Prints a PDF table and graph
    '''
    n = ser.size
    ser = ser.sort_values()
    ser.index = range(n)
    new_n = int(n-n*tail)
    plot_vals = ser[0:new_n]
    i = 0
    j = bin_width
    plt.figure().set_facecolor('w'); plt.gca().set_facecolor('w')
    to_print = []
    while j < plot_vals.max():
        prob = plot_vals[(plot_vals >= i)&(plot_vals < j)].count()/n
        to_print.append(f"P({i:>5.0f}<=X<={j:>5.0f}) = {prob:0.4f}")
        plt.bar(i, prob, width = bin_width, color = 'xkcd:ocean blue', edgecolor = 'black')
        i = j
        j += bin_width
    plt.bar(i, plot_vals[(plot_vals >= i)&(plot_vals < j)].count()/n, width = bin_width, color = 'xkcd:ocean blue', edgecolor = 'black')
    plt.bar(j, tail, width = bin_width, color = 'xkcd:ocean blue', edgecolor = 'black')
    to_print.append(f"        P(X>{i:.0f}) = {tail:0.4f}")
    plt.xticks([i for i in range(0,j+bin_width+1,2500)])
    plt.title(input('What would you like to name this graph?'), size = 16)
    plt.ylabel('Probablilty', size = 13); plt.xlabel('X', size = 13)
    plt.show()
    for x in to_print: print(x)

histogram()

What would you like to name this graph? Application

P(    0<=X<= 2000) = 0.5856
P( 2000<=X<= 4000) = 0.1828
P( 4000<=X<= 6000) = 0.0901
P( 6000<=X<= 8000) = 0.0515
P( 8000<=X<=10000) = 0.0322
P(10000<=X<=12000) = 0.0142
P(12000<=X<=14000) = 0.0167
P(14000<=X<=16000) = 0.0167
        P(X>16000) = 0.0090


### Part 2

In [13]:
def gauss_k(u):
    '''
    The Gaussian Kernel function
    Inputs:
        u - input to Gaussian kernel estimator
    Outputs:
        Returns value from Gaussian kernel estimator
    '''
    return (1/np.sqrt(2*np.pi))*np.exp((-1/2)*u**2)

def f_hHat(x, data, h):
    '''
    Uses the given formula to return an estimate of f(x) using the kernel density estimator formula
    Inputs:
        x - value estimating at
        data - ordered data to use for calculation
        h - bandwidth
    Outputs:
        Returns the estimate of f_h(x)
    '''
    n = data.size
    vals = []
    for i in range(n): vals.append(gauss_k((x-data[i])/h))
    return sum(vals)/(n*h)

def kernelDensityEstimator(ser = data['Apps'], h = 1000):
    '''
    Plots the kernel density estimator function
    Inputs:
        ser - data
        h - bandwidth
    Outputs:
        No return, plots the Kernel Density Estimator
    '''
    xVals = np.linspace(0,ser.max(), h)
    yVals = []
    ser = ser.sort_values()
    for x in xVals: yVals.append(f_hHat(x, ser, h))
    plt.figure().set_facecolor('w'); plt.gca().set_facecolor('w')
    plt.title("Guassian Kernel Density Estimator")
    plt.xlabel("Applications")
    plt.ylabel("Density")
    plt.plot(xVals, yVals)

kernelDensityEstimator()